# Predicting octane numbers from NIR spectra

In this example, we will predict [octane numbers](https://en.wikipedia.org/wiki/Octane_rating) from measured NIR spectra. Specifically, we will compare a least squares model to a partial least squares model and we will
use cross-validation to check the performance of the models.

## Reading the raw data

The raw data can be found in the file [gasoline.csv](./gasoline.csv). We will here load it with pandas and
extract the octane number, the spectra, and the wavelengths for the spectra.

In [ ]:
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

plt.style.use("seaborn-notebook")
sns.set_theme(style="ticks", context="notebook", palette="muted")
%matplotlib notebook

np.random.seed(2022)  # Set random state to get the same results for each run

In [ ]:
# Load the data set:
data = pd.read_csv("gasoline.csv")
# Each row contain a measured spectrum and a corresponding octane number:
# - extract the octane numbers:
yvars = ["octane"]
octane = data[yvars].to_numpy()
# - extract the spectra:
xvars = [i for i in data.columns if i not in yvars]
spectra = data[xvars].to_numpy()
# - get the wavelengths:
wavelengths = np.array([int(i.split()[0].split(".")[1]) for i in xvars])
print(f"Number of wavelengths measured: {len(xvars)}")
data.describe()

In [ ]:
# Let us visualize the spectra, just to see what we have to work with:
# We add some color, so we can color the spectra according to the octane numbers:
norm = mpl.colors.Normalize(vmin=octane.min(), vmax=octane.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap="Spectral")
fig, axi = plt.subplots(constrained_layout=True)
axi.set_title("NIR spectra")
for i, speci in enumerate(spectra):
    axi.plot(wavelengths, speci, color=cmap.to_rgba(octane[i]))
axi.set(xlabel="Wavelength (nm)", ylabel="Absorbance")
axi.set_facecolor("0.975")
fig.colorbar(cmap, label="Octane numbers")
sns.despine(fig=fig)

From the figure above, we see that there is some difference in the spectra, depending on the octane number. So, let us see if we can use this to create some models for predicting the octane numbers.

## Least squares and PLS with test-train and cross-validation

In [ ]:
from sklearn.preprocessing import scale
from sklearn.model_selection import (
    train_test_split,
    cross_validate,
    GridSearchCV,
)

X = scale(spectra)
Y = scale(octane)
# Create training and test sets:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [ ]:
# Create a PLS model, but use cross-validation to find the best number
# of pls components to use:
from sklearn.cross_decomposition import PLSRegression

parameters = {
    "n_components": range(1, 11),
}

grid = GridSearchCV(
    PLSRegression(),
    parameters,
    # scoring='r2',
    scoring="neg_mean_squared_error",
    refit=True,
)

grid.fit(X_train, Y_train)
print(grid.best_params_)
pls_model_optimized = grid.best_estimator_
B_PLS = pls_model_optimized.coef_[:, 0]

In [ ]:
fig, axi = plt.subplots(constrained_layout=True)
score = grid.cv_results_["mean_test_score"]
score_std = grid.cv_results_["std_test_score"]
axi.errorbar(
    parameters["n_components"],
    score,
    yerr=score_std,
    marker="o",
    markersize=12,
)
axi.set_xticks(parameters["n_components"])
axi.set(
    xlabel="PLS components", ylabel="score", title="Results from grid search"
)
sns.despine(fig=fig)

In [ ]:
# Create a linear model and train it:
from sklearn.linear_model import LinearRegression

leastsquares = LinearRegression(fit_intercept=False)
leastsquares.fit(X_train, Y_train)
B_MLR = leastsquares.coef_[0, :]

In [ ]:
# Create a LASSO model and train it:
from sklearn.linear_model import Lasso

parameters_lasso = {"alpha": np.logspace(-3, 2, 10)}

grid_lasso = GridSearchCV(
    Lasso(fit_intercept=False, max_iter=10000),
    parameters_lasso,
    scoring="neg_mean_squared_error",
    refit=True,
)
grid_lasso.fit(X_train, Y_train)
print(grid_lasso.best_params_)
lasso_model_optimized = grid_lasso.best_estimator_
B_lasso = lasso_model_optimized.coef_

In [ ]:
# Check performance with cross-validation:
fig, axes = plt.subplots(
    constrained_layout=True, ncols=3, sharex=True, sharey=True, figsize=(9, 3)
)

axes[0].set_title("Partial least squares", loc="left")
axes[1].set_title("Least squares", loc="left")
axes[2].set_title("Lasso", loc="left")

models = [pls_model_optimized, leastsquares, lasso_model_optimized]

for i, model in enumerate(models):
    cv_results = cross_validate(
        model,
        X_train,
        Y_train,
        scoring=("r2", "neg_mean_squared_error"),
        return_train_score=True,
    )
    axes[i].plot(cv_results["train_r2"], label="Training", marker="o")
    axes[i].plot(cv_results["test_r2"], label="Test", marker="o")
    axes[i].set_xlabel("CV no.")
axes[0].set_ylabel("R²")
axes[0].legend()
sns.despine(fig=fig)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error


def get_scores(y, y_hat):
    r2 = r2_score(y, y_hat)
    rmse = mean_squared_error(y, y_hat, squared=False)
    return r2, rmse


def add_scores(model, X_train, y_train, X_test, y_test, ax):
    """Plot y vs y_hat for test and training."""
    y_hat_train = model.predict(X_train)
    y_hat_test = model.predict(X_test)

    r2_train, rmsec = get_scores(y_train, y_hat_train)
    r2_test, rmsep = get_scores(y_test, y_hat_test)

    txt_train = f"R²(train) = {r2_train:6.4f}\nRMSEC = {rmsec:6.4f}"
    txt_test = f"R²(test) = {r2_test:6.4f}\nRMSEP = {rmsep:6.4f}"

    ax.scatter(Y_train, y_hat_train, label=txt_train)
    ax.scatter(Y_test, y_hat_test, label=txt_test)
    ax.legend()

In [ ]:
fig, axes = plt.subplots(
    constrained_layout=True,
    ncols=3,
    sharex=True,
    sharey=True,
    figsize=(9, 3),
)

axes[0].set_title("Partial least squares", loc="left")
axes[1].set_title("Least squares", loc="left")
axes[2].set_title("Lasso", loc="left")

models = [pls_model_optimized, leastsquares, lasso_model_optimized]

for i, model in enumerate(models):
    axes[i].set_aspect("equal")
    add_scores(model, X_train, Y_train, X_test, Y_test, axes[i])
    axes[i].set_xlabel("y")
axes[0].set_ylabel("ŷ")
sns.despine(fig=fig)

In [ ]:
fig, axes = plt.subplots(
    constrained_layout=True,
    ncols=1,
    nrows=3,
    sharex=True,
)

axes[0].set_title("Partial least squares", loc="left")
axes[1].set_title("Least squares", loc="left")
axes[2].set_title("Lasso", loc="left")

coeffs = [B_PLS, B_MLR, B_lasso]

for i, coef in enumerate(coeffs):
    axes[i].plot(wavelengths, coef)
    axes[i].axhline(y=0.0, ls=":", color="k")
    axes[i].set_ylabel("Coeff.")
sns.despine(fig=fig)

In [ ]:
norm = mpl.colors.Normalize(vmin=octane.min(), vmax=octane.max())
cmap = mpl.cm.ScalarMappable(norm=norm, cmap="Spectral")
fig, axi = plt.subplots(constrained_layout=True)
axi.set_title("NIR spectra")
for i, speci in enumerate(spectra):
    axi.plot(wavelengths, speci, color=cmap.to_rgba(octane[i]))
# axi.plot(wavelengths, B_lasso)
axi.set(xlabel="Wavelength (nm)", ylabel="Absorbance")
axi.set_facecolor("0.975")
fig.colorbar(cmap, label="Octane numbers")
sns.despine(fig=fig)